In [0]:
!pip install Faker

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType
from faker import Faker
import random

# Crear sesión Spark
spark = SparkSession.builder.appName("FakerComplexData").getOrCreate()

fake = Faker("es_ES")

data = []
for i in range(10):
    direccion = {
        "calle": fake.street_name(),
        "ciudad": fake.city(),
        "pais": fake.country()
    }

    # 🔁 Generar números base
    base_phones = [fake.phone_number() for _ in range(random.randint(1, 2))]

    # 🔁 Repetir algunos números y agregar nulos
    telefonos = []
    for _ in range(random.randint(1, 5)):
        choice = random.choice(["repeat", "null", "new"])
        if choice == "repeat":
            telefonos.append(random.choice(base_phones))
        elif choice == "null":
            telefonos.append(None)
        else:
            telefonos.append(fake.phone_number())

    # 🛒 Compras
    compras = [
        {"producto": fake.word(), "precio": round(random.uniform(10, 500), 2)}
        for _ in range(random.randint(1, 4))
    ]

    data.append({
        "id": i + 1,
        "nombre": fake.name(),
        "edad": random.randint(18, 70),
        "direccion": direccion,
        "telefonos": telefonos,
        "compras": compras
    })

# 🧱 Esquema del DataFrame
schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("nombre", StringType(), True),
    StructField("edad", IntegerType(), True),
    StructField("direccion", StructType([
        StructField("calle", StringType(), True),
        StructField("ciudad", StringType(), True),
        StructField("pais", StringType(), True)
    ])),
    StructField("telefonos", ArrayType(StringType())),
    StructField("compras", ArrayType(
        StructType([
            StructField("producto", StringType(), True),
            StructField("precio", DoubleType(), True)
        ])
    ))
])

# 🚀 Crear el DataFrame
df = spark.createDataFrame(data, schema=schema)

df.createOrReplaceTempView("customer_shop")

In [0]:
old_orders = [
    {"_id":1,"product":"Televisor","quantity":1,"price":345.32},
    {"_id":2,"product":"Televisor","quantity":1,"price":345.32},
    {"_id":3,"product":"Televisor","quantity":1,"price":345.32},
    {"_id":4,"product":"Televisor","quantity":1,"price":345.32},
    {"_id":5,"product":"Televisor","quantity":1,"price":345.32}
]